# 출력층 설계 (Output layer)

In [1]:
!pip3 install torch torchvision torchaudio

### 소프트맥스 오버플로우 방지

- ✅ 오버플로우(Overflow)란?
- 컴퓨터가 다룰 수 있는 수의 범위를 넘어서 버리는 현상
- 🧠 쉽게 말하면:
- 너무 너무 큰 숫자를 계산하려다가 컴퓨터가 "못해!" 하고 터지는 것

- ✅ 먼저 Softmax란?
- 벡터 값을 확률 분포처럼 바꿔주는 함수야.
- 각 값을 지수(exp)로 바꾸고, 전체 합으로 나눠서 합이 1이 되도록 정규화함.

In [2]:
import numpy as np

def softmax(x):
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x)

def stable_softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / np.sum(exp_x)


x = np.array([1000, 1001, 1002])

print(softmax(x))
print(stable_softmax(x))

[nan nan nan]
[0.09003057 0.24472847 0.66524096]


C:\Users\hyuna\AppData\Local\Temp\ipykernel_17548\3860681584.py:4: RuntimeWarning: overflow encountered in exp
  exp_x = np.exp(x)
C:\Users\hyuna\AppData\Local\Temp\ipykernel_17548\3860681584.py:5: RuntimeWarning: invalid value encountered in divide
  return exp_x / np.sum(exp_x)


- ✅ 왜 - np.max(x) 하는 거야?
- 수학적으로는 결과에 영향을 안 줘
- 왜냐하면 softmax는 어차피 이렇게 계산되니까:

$$
\text{softmax}(x_i) = \frac{e^{x_i}}{\sum_j e^{x_j}} = \frac{e^{x_i - C}}{\sum_j e^{x_j - C}}
$$

- 👉 어떤 값을 전체에서 같이 빼도 결과는 같아
- 그래서 np.max(x)를 빼서 계산을 안정화(overflow 방지) 하는 거야!

---

- PyTorch 라이브러리 함수 사용

In [3]:
import torch
import torch.nn.functional as F

x = torch.tensor([1000, 1001, 1002], dtype=torch.float32)

softmax_output = F.softmax(x, dim=0)    # dim: softmax를 적용할 축
print(softmax_output)

sigmoid_output = torch.sigmoid(x)
print(sigmoid_output)

tensor([0.0900, 0.2447, 0.6652])
tensor([1., 1., 1.])


### 손실 함수와 연계

In [4]:
# 신경망 생성
# torch.nn 패키지 사용
# nn.Module을 상속받고, 해당 모듈은 계층과 output을 반환하는 forward메소드를 포함
# 파이토치에서 신경망생성을 위한 기본 틀
# 1. class Net(nn.Module):

#     def __init__(self):
#         super(Net, self).__init__()

# 2. class MyModel(nn.Module):

#     def __init__(self):
#         super(MyModel, self).__init__()
# conv2d 에이어를 쌓을 때, 필터의 갯수를 계산하고 in_channels에 넣어줘야함

In [5]:
# "순전파 → 손실 계산 → 역전파 → 가중치 업데이트"

import torch
import torch.nn as nn
import torch.optim as optim

# 간단한 다중 클래스 분류 모델 정의 (입력 값: 5개, 출력: 3 클래스)
class SimpleMultiClassModel(nn.Module):
    def __init__(self):
        super(SimpleMultiClassModel, self).__init__()
        self.fc = nn.Linear(5, 3) # 입력 5개 → 출력 3개 (클래스 수), (즉, 클래스 3개 예측측)

    def forward(self, x):
        return self.fc(x)
    
model = SimpleMultiClassModel()
criterion = nn.CrossEntropyLoss() # → softmax + log + loss 계산까지 다 포함된 손실 함수, → 출력값은 softmax 안 써도 됨!!
# nn.CrossEntropyLoss()는 내부적으로 이렇게 계산함:log_softmax(output) + NLLLoss, → softmax 따로 쓰면 오히려 안 좋음 (중복됨)
optimizer = optim.Adam(model.parameters(), lr=0.01) # → 학습을 위한 가중치 최적화 알고리즘

# 데이터 생성
inputs = torch.randn(4, 5) # 샘플 4개, 각 샘플은 5개의 입력 특성
labels = torch.tensor([0, 2, 1, 0]) # 각 샘플의 정답 클래스 (정수로 표현)

for _ in range(10):
    preds = model(inputs)           # 순전파
    loss = criterion(preds, labels) # 손실 계산
    print(loss.item())

    optimizer.zero_grad()   # 이전 단계에서 계산된 기울기를 0으로 초기화
    loss.backward()         # 손실에 대한 역전파 수행 (파라미터에 대한 기울기 계산)
    optimizer.step()        # 계산된 기울기를 사용하여 옵티마이저가 모델의 파라미터 업데이트

1.1807864904403687
1.1498682498931885
1.1196541786193848
1.0901648998260498
1.0614186525344849
1.0334306955337524
1.0062131881713867
0.9797751307487488
0.9541224241256714
0.929257333278656
